In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
# Cài đặt Unsloth và các thư viện cần thiết
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
# CELL 3: Load base model and configure LoRA
from unsloth import FastLanguageModel
import torch

print("="*60)
print("🚀 LOADING MODEL & CONFIGURING LORA")
print("="*60)

# Configuration
max_seq_length = 1024
dtype = None
load_in_4bit = True
model_name = "unsloth/Qwen2.5-1.5B-Instruct"

# Load base model
print(f"\n⬇️ Loading {model_name}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
print("✅ Base model loaded!")

# Configure LoRA
print("\n🔧 Configuring LoRA adapters...")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)
print("✅ LoRA adapters configured!")
print(f"📊 Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

🚀 LOADING MODEL & CONFIGURING LORA

⬇️ Loading unsloth/Qwen2.5-1.5B-Instruct...
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Base model loaded!

🔧 Configuring LoRA adapters...
✅ LoRA adapters configured!
📊 Trainable parameters: 18,464,768


In [ ]:
# CELL 4: Load and format training data
from datasets import load_dataset

# System prompt (MUST match inference later!)
SYSTEM_PROMPT = """Bạn là một AI Router thông minh trong hệ thống tư vấn tâm lý.
Nhiệm vụ của bạn là phân loại câu nói của người dùng vào 1 trong 4 nhãn sau:

1. emotional_support: Cần an ủi, tâm sự, giải tỏa cảm xúc (VD: buồn, chán, cô đơn).
2. informational: Hỏi định nghĩa, kiến thức y khoa, thông tin thuốc/bệnh.
3. complex_consultation: Ca bệnh phức tạp, đa triệu chứng, cần suy luận sâu.
4. high_risk: Có ý định tự sát, tự hại, nguy hiểm tính mạng.

Chỉ trả về kết quả dưới dạng JSON duy nhất: {"label": "tên_nhãn"}"""

def formatting_prompts_func(examples):
    """Format data in ChatML style"""
    inputs = examples["text"]
    labels = examples["label"]
    texts = []

    for input_text, label in zip(inputs, labels):
        # ChatML format - CRITICAL!
        prompt = f"""<|im_start|>system
{SYSTEM_PROMPT}<|im_end|>
<|im_start|>user
{input_text}<|im_end|>
<|im_start|>assistant
{{"label": "{label}"}}<|im_end|>"""

        texts.append(prompt)

    return {"text": texts}

# Load training data
print("📂 Loading training data...")
train_file = "/content/drive/MyDrive/Major/Text_Reasoning/Text_Reasoning_train.jsonl"
train_dataset = load_dataset("json", data_files=train_file, split="train")
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)

print(f"✅ Loaded {len(train_dataset)} training samples")

# Show sample
print("\n📝 Sample formatted prompt:")
print("-"*60)
print(train_dataset[0]['text'][:350])
print("...")
print("-"*60)

📂 Loading training data...


Map:   0%|          | 0/5085 [00:00<?, ? examples/s]

✅ Loaded 5085 training samples

📝 Sample formatted prompt:
------------------------------------------------------------
<|im_start|>system
Bạn là một AI Router thông minh trong hệ thống tư vấn tâm lý.
Nhiệm vụ của bạn là phân loại câu nói của người dùng vào 1 trong 4 nhãn sau:

1. emotional_support: Cần an ủi, tâm sự, giải tỏa cảm xúc (VD: buồn, chán, cô đơn).
2. informational: Hỏi định nghĩa, kiến thức y khoa, thông tin thuốc/bệnh.
3. complex_consultation: Ca bệnh 
...
------------------------------------------------------------


In [ ]:
# CELL 5: Train the model
from trl import SFTTrainer
from transformers import TrainingArguments

print("="*60)
print("🚀 STARTING TRAINING")
print("="*60)

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
)

# Create trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

# Train!
print("\n🏃 Training started...")
trainer_stats = trainer.train()
print("\n🎉 Training completed!")

🚀 STARTING TRAINING


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/5085 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.



🏃 Training started...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,085 | Num Epochs = 1 | Total steps = 318
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 18,464,768 of 1,562,179,072 (1.18% trained)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kiennguyencodon1012 (kiennguyencodon1012-hanoi-university-of-industry) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.576900
20,0.765200
30,0.323000
40,0.257600
50,0.213800
60,0.172000
70,0.134100
80,0.115200
90,0.099100
100,0.093400



🎉 Training completed!


In [ ]:
# CELL 6: Save trained model
output_dir = "/content/drive/MyDrive/Major/Text_Reasoning/qwen_psychology_router_lora"

print(f"💾 Saving model to: {output_dir}")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("✅ Model saved successfully!")

# Verify files saved
import os
print("\n📂 Saved files:")
for f in os.listdir(output_dir):
    size = os.path.getsize(os.path.join(output_dir, f)) / (1024*1024)
    print(f"   {f:40} {size:>8.2f} MB")

💾 Saving model to: /content/drive/MyDrive/Major/Text_Reasoning/qwen_psychology_router_lora
✅ Model saved successfully!

📂 Saved files:
   README.md                                    0.01 MB
   adapter_model.safetensors                   70.49 MB
   adapter_config.json                          0.00 MB
   chat_template.jinja                          0.00 MB
   tokenizer_config.json                        0.00 MB
   special_tokens_map.json                      0.00 MB
   added_tokens.json                            0.00 MB
   vocab.json                                   2.65 MB
   merges.txt                                   1.59 MB
   tokenizer.json                              10.89 MB


In [ ]:
# CELL 7: Quick sanity check (3 predictions)
from unsloth import FastLanguageModel
import json

print("="*60)
print("🧪 QUICK SANITY CHECK")
print("="*60)

# Set to inference mode
FastLanguageModel.for_inference(model)

def quick_predict(text):
    """Quick prediction function"""
    prompt = f"""<|im_start|>system
{SYSTEM_PROMPT}<|im_end|>
<|im_start|>user
{text}<|im_end|>
<|im_start|>assistant
"""

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=64, temperature=0.1, do_sample=False)
    result = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Extract assistant response
    try:
        response = result.split("<|im_start|>assistant")[-1].split("<|im_end|>")[0].strip()
        parsed = json.loads(response)
        return parsed.get("label", "unknown")
    except:
        return "parse_error"

# Test
test_cases = [
    ("Em buồn quá, vừa chia tay", "emotional_support"),
    ("Trầm cảm là gì?", "informational"),
    ("Em muốn chết rồi", "high_risk"),
]

print("\n")
for text, expected in test_cases:
    pred = quick_predict(text)
    status = "✅" if pred == expected else "❌"
    print(f"{status} {text:40} → {pred:25} (expected: {expected})")

print("\n⚠️ If all 3 passed, proceed to full evaluation!")
print("   If failed, check format or retrain.")

🧪 QUICK SANITY CHECK


✅ Em buồn quá, vừa chia tay                → emotional_support         (expected: emotional_support)
✅ Trầm cảm là gì?                          → informational             (expected: informational)
✅ Em muốn chết rồi                         → high_risk                 (expected: high_risk)

⚠️ If all 3 passed, proceed to full evaluation!
   If failed, check format or retrain.


In [ ]:
# CELL 8: Load trained model for evaluation
from unsloth import FastLanguageModel
from peft import PeftModel
import torch

print("="*60)
print("🔄 LOADING MODEL FOR EVALUATION")
print("="*60)

# Paths
BASE_MODEL = "unsloth/Qwen2.5-1.5B-Instruct"
ADAPTER_PATH = "/content/drive/MyDrive/Major/Text_Reasoning/qwen_psychology_router_lora"

# Load base model
print(f"📥 Loading base model: {BASE_MODEL}")
eval_model, eval_tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL,
    max_seq_length=1024,
    dtype=None,
    load_in_4bit=True,
)

# Load LoRA adapter
print(f"📥 Loading LoRA adapter: {ADAPTER_PATH}")
eval_model = PeftModel.from_pretrained(eval_model, ADAPTER_PATH)

# Set inference mode
FastLanguageModel.for_inference(eval_model)

print("✅ Model loaded and ready for evaluation!")

🔄 LOADING MODEL FOR EVALUATION
📥 Loading base model: unsloth/Qwen2.5-1.5B-Instruct
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
📥 Loading LoRA adapter: /content/drive/MyDrive/Major/Text_Reasoning/qwen_psychology_router_lora
✅ Model loaded and ready for evaluation!


In [ ]:
# CELL 9: Define prediction function for evaluation
import json
import torch

# System prompt (MUST be same as training!)
SYSTEM_PROMPT = """Bạn là một AI Router thông minh trong hệ thống tư vấn tâm lý.
Nhiệm vụ của bạn là phân loại câu nói của người dùng vào 1 trong 4 nhãn sau:

1. emotional_support: Cần an ủi, tâm sự, giải tỏa cảm xúc (VD: buồn, chán, cô đơn).
2. informational: Hỏi định nghĩa, kiến thức y khoa, thông tin thuốc/bệnh.
3. complex_consultation: Ca bệnh phức tạp, đa triệu chứng, cần suy luận sâu.
4. high_risk: Có ý định tự sát, tự hại, nguy hiểm tính mạng.

Chỉ trả về kết quả dưới dạng JSON duy nhất: {"label": "tên_nhãn"}"""

def predict_intent(text):
    """
    Predict intent for a single query
    Returns: predicted_label (str)
    """
    # Build ChatML prompt
    prompt = f"""<|im_start|>system
{SYSTEM_PROMPT}<|im_end|>
<|im_start|>user
{text}<|im_end|>
<|im_start|>assistant
"""

    # Tokenize
    inputs = eval_tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate
    with torch.no_grad():
        outputs = eval_model.generate(
            **inputs,
            max_new_tokens=64,
            temperature=0.1,
            do_sample=False,
            use_cache=True,
            pad_token_id=eval_tokenizer.eos_token_id
        )

    # Decode
    full_output = eval_tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Extract assistant response
    try:
        assistant_response = full_output.split("<|im_start|>assistant")[-1]
        assistant_response = assistant_response.split("<|im_end|>")[0].strip()
    except:
        assistant_response = full_output.split("assistant")[-1].strip()

    # Parse JSON
    try:
        result = json.loads(assistant_response)
        return result.get("label", "unknown")
    except:
        # Fallback: search for label
        for label in ["emotional_support", "informational", "complex_consultation", "high_risk"]:
            if label in assistant_response.lower():
                return label
        return "unknown"

print("✅ Prediction function defined!")

✅ Prediction function defined!


In [ ]:
# CELL 10: Evaluate on full test set
import json
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

print("="*60)
print("📊 FULL EVALUATION ON TEST SET")
print("="*60)

# Load test data
test_file = "/content/drive/MyDrive/Major/Text_Reasoning/Text_Reasoning_test.jsonl"

print(f"📂 Loading test data: {test_file}")
with open(test_file, 'r', encoding='utf-8') as f:
    test_samples = [json.loads(line) for line in f]

print(f"📊 Total test samples: {len(test_samples)}")

# Run predictions
y_true = []
y_pred = []

print("\n🚀 Running predictions...")
for sample in tqdm(test_samples):
    text = sample['text']
    true_label = sample['label']

    pred_label = predict_intent(text)

    y_true.append(true_label)
    y_pred.append(pred_label)

# Calculate metrics
print("\n" + "="*60)
print("🏆 EVALUATION RESULTS")
print("="*60)

acc = accuracy_score(y_true, y_pred)
print(f"\n✅ Accuracy: {acc:.4f} ({acc*100:.2f}%)")
print("-" * 60)

print("\n📄 Classification Report:")
print(classification_report(y_true, y_pred, digits=4, zero_division=0))

# Confusion matrix
print("\n📊 Confusion Matrix:")
labels = ["emotional_support", "informational", "complex_consultation", "high_risk"]
cm = confusion_matrix(y_true, y_pred, labels=labels)

print("\n                    Predicted →")
print("True ↓        Emotional   Info   Complex   HighRisk")
for i, label in enumerate(["Emotional", "Info", "Complex", "HighRisk"]):
    print(f"{label:12}", end="  ")
    for j in range(4):
        print(f"{cm[i][j]:8}", end="  ")
    print()

# Sample errors
print("\n" + "="*60)
print("🔍 SAMPLE ERRORS (First 10)")
print("="*60)

error_count = 0
for i in range(len(y_true)):
    if y_true[i] != y_pred[i]:
        print(f"\n❌ Error #{error_count + 1}")
        print(f"   Input: {test_samples[i]['text'][:70]}...")
        print(f"   True:  {y_true[i]:25}")
        print(f"   Pred:  {y_pred[i]:25}")
        error_count += 1
        if error_count >= 10:
            break

print("\n" + "="*60)
print(f"✅ FINAL ACCURACY: {acc*100:.2f}%")
print("="*60)

📊 FULL EVALUATION ON TEST SET
📂 Loading test data: /content/drive/MyDrive/Major/Text_Reasoning/Text_Reasoning_test.jsonl
📊 Total test samples: 565

🚀 Running predictions...


100%|██████████| 565/565 [06:51<00:00,  1.37it/s]


🏆 EVALUATION RESULTS

✅ Accuracy: 1.0000 (100.00%)
------------------------------------------------------------

📄 Classification Report:
                      precision    recall  f1-score   support

complex_consultation     1.0000    1.0000    1.0000       120
   emotional_support     1.0000    1.0000    1.0000       252
           high_risk     1.0000    1.0000    1.0000        87
       informational     1.0000    1.0000    1.0000       106

            accuracy                         1.0000       565
           macro avg     1.0000    1.0000    1.0000       565
        weighted avg     1.0000    1.0000    1.0000       565


📊 Confusion Matrix:

                    Predicted →
True ↓        Emotional   Info   Complex   HighRisk
Emotional          252         0         0         0  
Info                 0       106         0         0  
Complex              0         0       120         0  
HighRisk             0         0         0        87  

🔍 SAMPLE ERRORS (First 10)

✅ FINAL